# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (7,7,1) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.999999
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=self.state_size, activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))       
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state, stochastic=False):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)[0]
        
        if stochastic:
            act_probs = np.exp(act_values)/np.exp(act_values).sum()
            return np.random.choice(np.arange(self.action_size), size=1, p=act_probs)[0]
              
        return np.argmax(act_values) 

    def replay(self): 
        if len(self.replay_buffer) < self.batch_size:
            return
        
        if len(self.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:    
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
        self.update_target_model()        
    
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame[:, :, 0]   
    f = f.reshape(7, 16, 7, 16).max(axis=(1, 3))
    #f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

#### Test Suite

In [10]:
def test_agent(stochastic=False):
    current_epsilon = agent.epsilon
    agent.epsilon = 0.0
    num_solved = 0
    solved_in_steps = defaultdict(int)

    for t in tqdm(range(100)):    
        random.seed(t)
        sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
        sok.set_maxsteps(20)
        steps = 0

        state = sok.get_image('rgb_array')
        done = False
        while not done:
            steps += 1
            action = agent.act(process_frame(state), stochastic)
            if action < 4:
                action += 1
            else:
                action += 5
            state, reward, done, info = sok.step(action)

        if 3 in sok.room_state:            
            num_solved += 1
            solved_in_steps[steps] += 1
    
    agent.epsilon = current_epsilon
    print("*" * 30)
    print("Stochastic" if stochastic else "Deterministic")
    print("*" * 30)
    print("Solved: %d" % num_solved)
    print("=" * 30)
    print(solved_in_steps)
    print("*" * 30)

In [11]:
max_episodes = 20000
max_steps = 20

def init_sok(r):
    random.seed(r+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [12]:
agent = SOK_Agent()

steps_per_episode = []

for e in range(max_episodes):
    if e % 100 == 0 and e > 0:
        test_agent(stochastic=False)
        
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    random.seed(e)
    
    state = process_frame(sok.get_image('rgb_array'))
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()                
        
        if (step+1) % agent.replay_rate == 0:
            agent.replay()            
        
        if done:   
            steps_per_episode.append(step+1)
            
            if 3 in sok.room_state:                
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(step+1)
            
            #agent.save("exp1_episode%d.h5" % (e))            
            break

Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 6 Epsilon 1.0000
Episode: 8
Episode: 9
Episode: 10
SOLVED! Episode 10 Steps: 11 Epsilon 1.0000
Episode: 11
Episode: 12
Episode: 13
Episode: 14
Episode: 15
Episode: 16
Episode: 17
Episode: 18
Episode: 19
Episode: 20
Episode: 21
Episode: 22
Episode: 23
Episode: 24
Episode: 25
Episode: 26
SOLVED! Episode 26 Steps: 1 Epsilon 1.0000
Episode: 27
Episode: 28
Episode: 29
Episode: 30
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39
Episode: 40
Episode: 41
Episode: 42
Episode: 43
Episode: 44
Episode: 45
Episode: 46
Episode: 47
SOLVED! Episode 47 Steps: 17 Epsilon 0.9999
Episode: 48
Episode: 49
Episode: 50
Episode: 51
Episode: 52
Episode: 53
Episode: 54
Episode: 55
Episode: 56
SOLVED! Episode 56 Steps: 13 Epsilon 0.9999
Episode: 57
Episode: 58
Episode: 59
SOLVED! Episode 59 Steps: 7 Epsilon 0.9999
Episode: 60
Episode: 61
SOLVED


******************************
Deterministic
******************************
Solved: 17
defaultdict(<class 'int'>, {3: 8, 1: 6, 2: 3})
******************************
Episode: 100
Episode: 101
Episode: 102
Episode: 103
Episode: 104
Episode: 105
Episode: 106
Episode: 107
Episode: 108
Episode: 109
Episode: 110
Episode: 111
Episode: 112
Episode: 113
Episode: 114
SOLVED! Episode 114 Steps: 6 Epsilon 0.9998
Episode: 115
Episode: 116
Episode: 117
Episode: 118
Episode: 119
Episode: 120
Episode: 121
Episode: 122
Episode: 123
Episode: 124
Episode: 125
SOLVED! Episode 125 Steps: 18 Epsilon 0.9998
Episode: 126
Episode: 127
Episode: 128
Episode: 129
SOLVED! Episode 129 Steps: 2 Epsilon 0.9998
Episode: 130
Episode: 131
Episode: 132
Episode: 133
Episode: 134
Episode: 135
Episode: 136
SOLVED! Episode 136 Steps: 8 Epsilon 0.9998
Episode: 137
Episode: 138
Episode: 139
Episode: 140
Episode: 141
SOLVED! Episode 141 Steps: 7 Epsilon 0.9997
Episode: 142
Episode: 143
Episode: 144
Episode: 145
Episode: 146
Ep


******************************
Deterministic
******************************
Solved: 17
defaultdict(<class 'int'>, {2: 3, 1: 10, 3: 4})
******************************
Episode: 200
Episode: 201
Episode: 202
Episode: 203
Episode: 204
Episode: 205
Episode: 206
Episode: 207
Episode: 208
Episode: 209
SOLVED! Episode 209 Steps: 13 Epsilon 0.9996
Episode: 210
Episode: 211
Episode: 212
Episode: 213
Episode: 214
SOLVED! Episode 214 Steps: 12 Epsilon 0.9996
Episode: 215
Episode: 216
Episode: 217
Episode: 218
Episode: 219
Episode: 220
SOLVED! Episode 220 Steps: 1 Epsilon 0.9996
Episode: 221
SOLVED! Episode 221 Steps: 1 Epsilon 0.9996
Episode: 222
Episode: 223
Episode: 224
Episode: 225
Episode: 226
Episode: 227
Episode: 228
Episode: 229
Episode: 230
SOLVED! Episode 230 Steps: 3 Epsilon 0.9996
Episode: 231
Episode: 232
Episode: 233
SOLVED! Episode 233 Steps: 9 Epsilon 0.9996
Episode: 234
Episode: 235
SOLVED! Episode 235 Steps: 19 Epsilon 0.9996
Episode: 236
Episode: 237
SOLVED! Episode 237 Steps: 6


******************************
Deterministic
******************************
Solved: 19
defaultdict(<class 'int'>, {3: 7, 2: 1, 1: 11})
******************************
Episode: 300
SOLVED! Episode 300 Steps: 3 Epsilon 0.9995
Episode: 301
Episode: 302
Episode: 303
Episode: 304
Episode: 305
Episode: 306
Episode: 307
SOLVED! Episode 307 Steps: 10 Epsilon 0.9994
Episode: 308
Episode: 309
Episode: 310
Episode: 311
Episode: 312
Episode: 313
Episode: 314
Episode: 315
Episode: 316
Episode: 317
Episode: 318
Episode: 319
Episode: 320
Episode: 321
Episode: 322
SOLVED! Episode 322 Steps: 2 Epsilon 0.9994
Episode: 323
Episode: 324
Episode: 325
Episode: 326
Episode: 327
SOLVED! Episode 327 Steps: 8 Epsilon 0.9994
Episode: 328
Episode: 329
Episode: 330
SOLVED! Episode 330 Steps: 12 Epsilon 0.9994
Episode: 331
Episode: 332
Episode: 333
Episode: 334
SOLVED! Episode 334 Steps: 1 Epsilon 0.9994
Episode: 335
Episode: 336
Episode: 337
Episode: 338
Episode: 339
Episode: 340
Episode: 341
Episode: 342
Episode:


******************************
Deterministic
******************************
Solved: 19
defaultdict(<class 'int'>, {1: 11, 2: 3, 3: 5})
******************************
Episode: 400
Episode: 401
Episode: 402
Episode: 403
Episode: 404
Episode: 405
Episode: 406
Episode: 407
Episode: 408
Episode: 409
Episode: 410
Episode: 411
Episode: 412
Episode: 413
Episode: 414
Episode: 415
Episode: 416
Episode: 417
Episode: 418
Episode: 419
Episode: 420
Episode: 421
Episode: 422
Episode: 423
Episode: 424
Episode: 425
Episode: 426
Episode: 427
Episode: 428
Episode: 429
SOLVED! Episode 429 Steps: 1 Epsilon 0.9992
Episode: 430
Episode: 431
Episode: 432
Episode: 433
Episode: 434
Episode: 435
SOLVED! Episode 435 Steps: 4 Epsilon 0.9992
Episode: 436
Episode: 437
Episode: 438
SOLVED! Episode 438 Steps: 2 Epsilon 0.9992
Episode: 439
Episode: 440
Episode: 441
SOLVED! Episode 441 Steps: 3 Epsilon 0.9992
Episode: 442
Episode: 443
SOLVED! Episode 443 Steps: 11 Epsilon 0.9992
Episode: 444
Episode: 445
Episode: 446
E

...


******************************
Deterministic
******************************
Solved: 36
defaultdict(<class 'int'>, {3: 14, 2: 9, 1: 13})
******************************
Episode: 11200
Episode: 11201
Episode: 11202
Episode: 11203
Episode: 11204
Episode: 11205
Episode: 11206
SOLVED! Episode 11206 Steps: 1 Epsilon 0.9798
Episode: 11207
Episode: 11208
Episode: 11209
Episode: 11210
Episode: 11211
Episode: 11212
Episode: 11213
Episode: 11214
Episode: 11215
Episode: 11216
Episode: 11217
Episode: 11218
Episode: 11219
Episode: 11220
Episode: 11221
Episode: 11222
Episode: 11223
Episode: 11224
Episode: 11225
Episode: 11226
Episode: 11227
SOLVED! Episode 11227 Steps: 3 Epsilon 0.9798
Episode: 11228
Episode: 11229
Episode: 11230
Episode: 11231
Episode: 11232
Episode: 11233
Episode: 11234
Episode: 11235
Episode: 11236
Episode: 11237
Episode: 11238
Episode: 11239
Episode: 11240
SOLVED! Episode 11240 Steps: 16 Epsilon 0.9798
Episode: 11241
Episode: 11242
Episode: 11243
Episode: 11244
Episode: 11245
SOL


******************************
Deterministic
******************************
Solved: 33
defaultdict(<class 'int'>, {3: 14, 2: 7, 1: 12})
******************************
Episode: 11300
Episode: 11301
Episode: 11302
Episode: 11303
Episode: 11304
Episode: 11305
Episode: 11306
Episode: 11307
Episode: 11308
Episode: 11309
SOLVED! Episode 11309 Steps: 5 Epsilon 0.9796
Episode: 11310
Episode: 11311
Episode: 11312
SOLVED! Episode 11312 Steps: 11 Epsilon 0.9796
Episode: 11313
Episode: 11314
Episode: 11315
Episode: 11316
Episode: 11317
SOLVED! Episode 11317 Steps: 4 Epsilon 0.9796
Episode: 11318
Episode: 11319
Episode: 11320
SOLVED! Episode 11320 Steps: 1 Epsilon 0.9796
Episode: 11321
Episode: 11322
Episode: 11323
SOLVED! Episode 11323 Steps: 6 Epsilon 0.9796
Episode: 11324
Episode: 11325
Episode: 11326
Episode: 11327
Episode: 11328
Episode: 11329
Episode: 11330
Episode: 11331
SOLVED! Episode 11331 Steps: 15 Epsilon 0.9796
Episode: 11332
Episode: 11333
Episode: 11334
Episode: 11335
SOLVED! Episod


******************************
Deterministic
******************************
Solved: 35
defaultdict(<class 'int'>, {3: 13, 2: 4, 1: 17, 5: 1})
******************************
Episode: 11400
Episode: 11401
Episode: 11402
Episode: 11403
SOLVED! Episode 11403 Steps: 18 Epsilon 0.9795
Episode: 11404
Episode: 11405
Episode: 11406
Episode: 11407
Episode: 11408
Episode: 11409
Episode: 11410
Episode: 11411
Episode: 11412
Episode: 11413
Episode: 11414
Episode: 11415
Episode: 11416
Episode: 11417
Episode: 11418
Episode: 11419
Episode: 11420
SOLVED! Episode 11420 Steps: 1 Epsilon 0.9794
Episode: 11421
SOLVED! Episode 11421 Steps: 13 Epsilon 0.9794
Episode: 11422
Episode: 11423
SOLVED! Episode 11423 Steps: 9 Epsilon 0.9794
Episode: 11424
Episode: 11425
Episode: 11426
Episode: 11427
Episode: 11428
Episode: 11429
SOLVED! Episode 11429 Steps: 7 Epsilon 0.9794
Episode: 11430
Episode: 11431
Episode: 11432
SOLVED! Episode 11432 Steps: 8 Epsilon 0.9794
Episode: 11433
SOLVED! Episode 11433 Steps: 11 Epsilo


******************************
Deterministic
******************************
Solved: 40
defaultdict(<class 'int'>, {3: 16, 2: 8, 1: 16})
******************************
Episode: 11500
Episode: 11501
Episode: 11502
Episode: 11503
Episode: 11504


KeyboardInterrupt: 